In [13]:
# Extract positive pairs for training
import os
import numpy as np
import PIL
from PIL import Image
import random

folders_train_set = ['set00', 'set01', 'set02', 'set03', 'set04', 'set05']
folders_v         = ['V000', 'V001', 'V002', 'V003', 'V004', 'V005', 'V006', 'V007', 'V008']

path = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/'
path_train_save = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/pose_train/'

frame_jump = 45
pixel_step = 12
patch_side = 32

cnt = 0
for folder1 in folders_train_set:
    for folder2 in folders_v:
        color_path   = '{}/{}/{}/visible'.format(path, folder1, folder2)
        thermal_path = '{}/{}/{}/lwir'.format(path, folder1, folder2)
        
        file_cnt = 0
        for pack in os.walk(color_path):
            for f in pack[2]:
                file_cnt += 1                
        
        for scene in range(0, file_cnt, frame_jump):
            color_img   = color_path + '/I{}.jpg'.format(str(scene).zfill(5))
            thermal_img = thermal_path + '/I{}.jpg'.format(str(scene).zfill(5))
        
            list_im = [thermal_img, color_img]
            imgs    = [PIL.Image.open(i) for i in list_im]
        
            W, H = imgs[0].size
        
            imgs[0] = imgs[0].convert("RGB")
            imgs[1] = imgs[1].convert("RGB")            
            
            # extract patches            
            for i in range(H//2-patch_side*2, H-patch_side*2, pixel_step):
                for j in range(patch_side, W-patch_side, pixel_step):                    
                    crop_th  = imgs[0].crop((j-patch_side, i-patch_side, j+patch_side, i+patch_side))
                    crop_clr = imgs[1].crop((j-patch_side, i-patch_side, j+patch_side, i+patch_side))
                        
                    pos = [crop_th, crop_clr]

                    imgs_comb = np.hstack((np.asarray(i) for i in pos))
                    imgs_comb = PIL.Image.fromarray(imgs_comb)
                    save_path = '{}/{}.jpg'.format(path_train_save, cnt)
                    imgs_comb.save(save_path)
                    cnt += 1            
            
print('Number of positive patches in training dataset: ', cnt)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


Number of positive patches in training dataset:  1192224


In [1]:
# Extract negative pairs from positive pairs for training
import random
import os
import numpy as np
import PIL
from PIL import Image

path_train      = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/pose_train'
path_save_train = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/neg_train'

file_cnt_train = 1192224
cnt = 0
positive_list = list( range(0, file_cnt_train) )
random.shuffle(positive_list)

for i in positive_list:
    ran_num = random.randint(0, file_cnt_train - 1)
    while ran_num == i:
        ran_num = random.randint(0, file_cnt_train - 1)
        
    th_patch_path  = '{}/{}.jpg'.format(path_train, ran_num)
    clr_patch_path = '{}/{}.jpg'.format(path_train, i)
    
    clr_patch = PIL.Image.open(clr_patch_path)
    th_patch  = PIL.Image.open(th_patch_path)

    neg_clr_crop = clr_patch.crop((64, 0, 128, 64))
    neg_th_crop  = th_patch.crop((0, 0, 64, 64))
    width2, height2 = neg_th_crop.size
    
    neg = [neg_th_crop, neg_clr_crop]
    
    imgs_comb = np.hstack((np.asarray(j) for j in neg))
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    save_path = '{}/{}.jpg'.format(path_save_train, cnt)
    imgs_comb.save(save_path)
    cnt += 1
    
print('Number of negative patches in training dataset: ', cnt)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


Number of negative patches in training dataset:  1192224


In [2]:
## Merge postive and negative pairs for training
import os
import numpy as np
import PIL
from PIL import Image

path_pos_train  = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/pose_train'
path_neg_train  = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/neg_train'
path_save_train = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/train_set'

file_cnt_pos_train = 1192224
for i in range(0, file_cnt_pos_train):
    pos_patch_path  = '{}/{}.jpg'.format(path_pos_train, i)
    neg_patch_path  = '{}/{}.jpg'.format(path_neg_train, i)
    pos_patch = PIL.Image.open(pos_patch_path)
    neg_patch = PIL.Image.open(neg_patch_path)
   
    merge = [pos_patch, neg_patch]
    
    imgs_comb = np.hstack((np.asarray(j) for j in merge))
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    save_path = '{}/{}.jpg'.format(path_save_train, i)
    imgs_comb.save(save_path)
    
print('Number of merged patches in training dataset')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


Number of merged patches in training dataset


In [8]:
# Extract positive pairs for testing
import os
import numpy as np
import PIL
from PIL import Image
import random

folders_test_set  = ['set06', 'set07', 'set08', 'set09', 'set10', 'set11']
folders_v         = ['V000', 'V001', 'V002', 'V003', 'V004', 'V005', 'V006', 'V007', 'V008']

path            = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal'
path_test_save  = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/pose_test'

frame_jump = 45
pixel_step = 12
patch_side = 32

for folder1 in folders_test_set:
    cnt = 0
    for folder2 in folders_v:
        color_path   = '{}/{}/{}/visible'.format(path, folder1, folder2)
        thermal_path = '{}/{}/{}/lwir'.format(path, folder1, folder2)
        
        file_cnt = 0
        for pack in os.walk(color_path):
            for f in pack[2]:
                file_cnt += 1
        
        print(folder1, '  - file_cnt: ', file_cnt)
               
        for scene in range(0, file_cnt, frame_jump):
            color_img   = color_path + '/I{}.jpg'.format(str(scene).zfill(5))
            thermal_img = thermal_path + '/I{}.jpg'.format(str(scene).zfill(5))
        
            list_im = [thermal_img, color_img]
            imgs    = [PIL.Image.open(i) for i in list_im]
        
            W, H = imgs[0].size
        
            imgs[0] = imgs[0].convert("RGB")
            imgs[1] = imgs[1].convert("RGB")            
            
            # extract patches
            for i in range(H//2-patch_side*2, H-patch_side*2, pixel_step):
                for j in range(patch_side, W-patch_side, pixel_step):                    
                    crop_th  = imgs[0].crop((j-patch_side, i-patch_side, j+patch_side, i+patch_side))
                    crop_clr = imgs[1].crop((j-patch_side, i-patch_side, j+patch_side, i+patch_side))
                        
                    pos = [crop_th, crop_clr]

                    imgs_comb = np.hstack((np.asarray(i) for i in pos))
                    imgs_comb = PIL.Image.fromarray(imgs_comb)
                    save_path    = '{}/{}/{}.jpg'.format(path_test_save, folder1, cnt)
                    imgs_comb.save(save_path)
                    cnt += 1            
            
    print('Number of positive patches in testing dataset: ', cnt)

set06   - file_cnt:  1523


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


set06   - file_cnt:  2691
set06   - file_cnt:  1688
set06   - file_cnt:  4285
set06   - file_cnt:  2800
set06   - file_cnt:  0
set06   - file_cnt:  0
set06   - file_cnt:  0
set06   - file_cnt:  0
Number of positive patches in testing dataset:  307296
set07   - file_cnt:  2544
set07   - file_cnt:  4000
set07   - file_cnt:  1597
set07   - file_cnt:  0
set07   - file_cnt:  0
set07   - file_cnt:  0
set07   - file_cnt:  0
set07   - file_cnt:  0
set07   - file_cnt:  0
Number of positive patches in testing dataset:  192192
set08   - file_cnt:  2578
set08   - file_cnt:  2972
set08   - file_cnt:  2500
set08   - file_cnt:  0
set08   - file_cnt:  0
set08   - file_cnt:  0
set08   - file_cnt:  0
set08   - file_cnt:  0
set08   - file_cnt:  0
Number of positive patches in testing dataset:  191136
set09   - file_cnt:  3500
set09   - file_cnt:  0
set09   - file_cnt:  0
set09   - file_cnt:  0
set09   - file_cnt:  0
set09   - file_cnt:  0
set09   - file_cnt:  0
set09   - file_cnt:  0
set09   - file_cnt: 

In [7]:
# Extract negative pairs from positive pairs for testing
import random
import os
import numpy as np
import PIL
from PIL import Image

path_train     = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/pose_test'
path_test_save = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/neg_test'

folders_test_set  = ['set06', 'set07', 'set08', 'set09', 'set10', 'set11']
folders_v         = ['V000', 'V001', 'V002', 'V003', 'V004', 'V005', 'V006', 'V007', 'V008']

file_sizes = [307296, 192192, 191136, 82368, 210144, 84480]

cnt = 0

for f_idx in range(6):
    file_cnt = file_sizes[f_idx]
    folder   = folders_test_set[f_idx] 
    
    print(folder, file_cnt)
    
    positive_list = list( range(0, file_cnt) )
    random.shuffle(positive_list)

    for i in positive_list:
        ran_num = random.randint(0, file_cnt - 1)
        while ran_num == i:
            ran_num = random.randint(0, file_cnt - 1)

        th_patch_path  = '{}/{}/{}.jpg'.format(path_train, folder, ran_num)
        clr_patch_path = '{}/{}/{}.jpg'.format(path_train, folder, i)

        clr_patch = PIL.Image.open(clr_patch_path)
        th_patch  = PIL.Image.open(th_patch_path)

        neg_clr_crop = clr_patch.crop((64, 0, 128, 64))
        neg_th_crop  = th_patch.crop((0, 0, 64, 64))
        width2, height2 = neg_th_crop.size

        neg = [neg_th_crop, neg_clr_crop]

        imgs_comb = np.hstack((np.asarray(j) for j in neg))
        imgs_comb = PIL.Image.fromarray(imgs_comb)
        save_path = '{}/{}/{}.jpg'.format(path_test_save, folder, cnt)
        imgs_comb.save(save_path)
        cnt += 1
    
print('Number of negative patches in testing dataset')

set06 307296


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


set07 192192
set08 191136
set09 82368
set10 210144
set11 84480
Number of negative patches in testing dataset


In [8]:
## Merge postive and negative pairs for testing
import os
import numpy as np
import PIL
from PIL import Image

path_pos_train  = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/pose_test'
path_neg_train  = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/neg_test'
path_test_save  = '/home/shared_dir/research/SPIMNet/KAIST_RGB_Thermal/processed/test_set'

file_sizes = [307296, 192192, 191136, 82368, 210144, 84480]

for f_idx in range(6):
    file_cnt = file_sizes[f_idx]
    folder   = folders_test_set[f_idx]  
    
    for i in range(file_cnt):
        pos_patch_path  = '{}/{}/{}.jpg'.format(path_pos_train, folder, i)
        neg_patch_path  = '{}/{}/{}.jpg'.format(path_neg_train, folder, i)
        pos_patch = PIL.Image.open(pos_patch_path)
        neg_patch = PIL.Image.open(neg_patch_path)

        merge = [pos_patch, neg_patch]

        imgs_comb = np.hstack((np.asarray(j) for j in merge))
        imgs_comb = PIL.Image.fromarray(imgs_comb)
        save_path = '{}/{}/{}.jpg'.format(path_test_save, folder, i)
        imgs_comb.save(save_path)
    
print('Number of merged patches in testing dataset')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


Number of merged patches in testing dataset
